In [1]:
from __future__ import print_function, division
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import torch.backends.cudnn as cudnn
import numpy as np
import torchvision
from torchvision import datasets, transforms
import matplotlib.pyplot as plt
import time
import os
import copy
import sys

In [2]:
from pathlib import Path
sys.path.insert(0,os.path.abspath('../'))
sys.path.insert(0,os.path.abspath('../utilities'))

In [3]:
from utilities.resnet import lambda_resnet38

In [4]:
from utilities.utils import *

## Lambda Resnet 38 on CIFAR-100

In [5]:
SEED=566
torch.manual_seed(SEED)
np.random.seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed(SEED)
    torch.cuda.manual_seed_all(SEED)

In [6]:
model = lambda_resnet38(num_classes=100)
PRETRAINED_MODEL_PATH=os.path.join('/home/abc586/courseWork/introtodlsystem/finalProject/visualizing-vision-models/pretrained-models','lambda_resnet38_seed_566_epochs_100.pth')
model.load_state_dict(torch.load(PRETRAINED_MODEL_PATH))

<All keys matched successfully>

In [7]:
LABEL='lambdar38-CIFAR100'
TYPE='PRETRAINED'

## Getting the hooks for each layers

In [8]:
hookLayersActivationDict = {}
activationDict = {}

In [9]:
modelNamedModules = dict(model.named_modules())

In [10]:
hookLayers = []
for i in modelNamedModules.keys():
    if i.__contains__("bn") or i.__contains__("relu") or i.__contains__("avgpool") or i.__contains__("fc") or i.__contains__("maxpool") :
        hookLayers.append(i)

In [11]:
for layer in hookLayers:
    h = model.get_submodule(layer).register_forward_hook(getActivation(layer,hookLayersActivationDict))

## CIFAR 100

In [12]:
TRAIN_BATCH_SIZE = 256
TEST_BATCH_SIZE = 256
NUM_WORKERS = 0

normalize = transforms.Normalize(mean=[0.5071, 0.4867, 0.4408],
                                     std=[0.2675, 0.2565, 0.2761])

train_dataset = datasets.CIFAR100(root='./data', train=True, transform=transforms.Compose([
            transforms.RandomHorizontalFlip(),
            transforms.RandomCrop(32, 4),
            transforms.ToTensor(),
            normalize,
        ]), download=True)

train_loader = torch.utils.data.DataLoader(train_dataset,
        batch_size=TRAIN_BATCH_SIZE, shuffle=True,
        num_workers=NUM_WORKERS, pin_memory=True)

val_loader = torch.utils.data.DataLoader(
        datasets.CIFAR100(root='./data', train=False, transform=transforms.Compose([
            transforms.ToTensor(),
            normalize,
        ])),
        batch_size=TEST_BATCH_SIZE, shuffle=False,
        num_workers=NUM_WORKERS, pin_memory=True)

Files already downloaded and verified


In [13]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [14]:
model = model.to(device)
valid_acc,activationDict = evaluate(model,device,val_loader,hookLayers,hookLayersActivationDict)

Iteration:   0%|          | 0/40 [00:01<?, ?it/s]


## HSIC calculation and plots

In [ ]:
col1,col2,hsicScoreList = getLayerWiseOutputCorrelation(hookLayers,hookLayers,activationDict,activationDict)

Layer 1:bn1, HSIC score:1.2190240316180647e-34
Layer 1:layer1.0.bn1, HSIC score:nan
Cross HSIC score:inf
Denom:nan
Layer 1:bn1, HSIC score:1.2190240316180647e-34
Layer 1:layer1.0.relu1, HSIC score:nan
Cross HSIC score:inf
Denom:nan
Layer 1:bn1, HSIC score:1.2190240316180647e-34
Layer 1:layer2.0.conv2.avgpool, HSIC score:nan
Cross HSIC score:inf
Denom:nan
Layer 1:relu, HSIC score:1.2190240316180647e-34
Layer 1:layer1.0.bn1, HSIC score:nan
Cross HSIC score:inf
Denom:nan
Layer 1:relu, HSIC score:1.2190240316180647e-34
Layer 1:layer1.0.relu1, HSIC score:nan
Cross HSIC score:inf
Denom:nan
Layer 1:relu, HSIC score:1.2190240316180647e-34
Layer 1:layer2.0.conv2.avgpool, HSIC score:nan
Cross HSIC score:inf
Denom:nan
Layer 1:layer1.0.bn1, HSIC score:nan
Layer 1:bn1, HSIC score:1.2190240316180647e-34
Cross HSIC score:nan
Denom:nan
Layer 1:layer1.0.bn1, HSIC score:nan
Layer 1:relu, HSIC score:1.2190240316180647e-34
Cross HSIC score:nan
Denom:nan
Layer 1:layer1.0.bn1, HSIC score:nan
Layer 1:layer1.

Layer 1:layer1.0.relu1, HSIC score:nan
Layer 1:layer1.1.relu3, HSIC score:8.725542964290456e-21
Cross HSIC score:nan
Denom:nan
Layer 1:layer1.0.relu1, HSIC score:nan
Layer 1:layer2.0.bn1, HSIC score:inf
Cross HSIC score:nan
Denom:nan
Layer 1:layer1.0.relu1, HSIC score:nan
Layer 1:layer2.0.relu1, HSIC score:inf
Cross HSIC score:nan
Denom:nan
Layer 1:layer1.0.relu1, HSIC score:nan
Layer 1:layer2.0.conv2.avgpool, HSIC score:nan
Cross HSIC score:nan
Denom:nan
Layer 1:layer1.0.relu1, HSIC score:nan
Layer 1:layer2.0.bn2, HSIC score:685302096068608.0
Cross HSIC score:nan
Denom:nan
Layer 1:layer1.0.relu1, HSIC score:nan
Layer 1:layer2.0.relu2, HSIC score:685302096068608.0
Cross HSIC score:nan
Denom:nan
Layer 1:layer1.0.relu1, HSIC score:nan
Layer 1:layer2.0.bn3, HSIC score:0.002746131969615817
Cross HSIC score:nan
Denom:nan
Layer 1:layer1.0.relu1, HSIC score:nan
Layer 1:layer2.0.relu3, HSIC score:0.002746131969615817
Cross HSIC score:nan
Denom:nan
Layer 1:layer1.0.relu1, HSIC score:nan
Layer 1

Layer 1:layer1.0.bn2, HSIC score:1.3307418100794166e-07
Layer 1:layer2.0.conv2.avgpool, HSIC score:nan
Cross HSIC score:inf
Denom:nan
Layer 1:layer1.0.relu2, HSIC score:1.3307418100794166e-07
Layer 1:layer1.0.bn1, HSIC score:nan
Cross HSIC score:inf
Denom:nan
Layer 1:layer1.0.relu2, HSIC score:1.3307418100794166e-07
Layer 1:layer1.0.relu1, HSIC score:nan
Cross HSIC score:inf
Denom:nan
Layer 1:layer1.0.relu2, HSIC score:1.3307418100794166e-07
Layer 1:layer2.0.conv2.avgpool, HSIC score:nan
Cross HSIC score:inf
Denom:nan
Layer 1:layer1.0.bn3, HSIC score:1.1212200765404163e-17
Layer 1:layer1.0.bn1, HSIC score:nan
Cross HSIC score:inf
Denom:nan
Layer 1:layer1.0.bn3, HSIC score:1.1212200765404163e-17
Layer 1:layer1.0.relu1, HSIC score:nan
Cross HSIC score:inf
Denom:nan
Layer 1:layer1.0.bn3, HSIC score:1.1212200765404163e-17
Layer 1:layer2.0.conv2.avgpool, HSIC score:nan
Cross HSIC score:inf
Denom:nan
Layer 1:layer1.0.relu3, HSIC score:1.1212200765404163e-17
Layer 1:layer1.0.bn1, HSIC score:n

/home/abc586/courseWork/introtodlsystem/finalProject/visualizing-vision-models/utilities/utils.py:104: RuntimeWarning: invalid value encountered in double_scalars
  finalScore = hsicCross/denom


Layer 1:layer2.0.relu1, HSIC score:inf
Layer 1:layer1.0.bn1, HSIC score:nan
Cross HSIC score:inf
Denom:nan
Layer 1:layer2.0.relu1, HSIC score:inf
Layer 1:layer1.0.relu1, HSIC score:nan
Cross HSIC score:inf
Denom:nan
Layer 1:layer2.0.relu1, HSIC score:inf
Layer 1:layer2.0.conv2.avgpool, HSIC score:nan
Cross HSIC score:inf
Denom:nan
Layer 1:layer2.0.conv2.avgpool, HSIC score:nan
Layer 1:bn1, HSIC score:1.2190240316180647e-34
Cross HSIC score:nan
Denom:nan
Layer 1:layer2.0.conv2.avgpool, HSIC score:nan
Layer 1:relu, HSIC score:1.2190240316180647e-34
Cross HSIC score:nan
Denom:nan
Layer 1:layer2.0.conv2.avgpool, HSIC score:nan
Layer 1:layer1.0.bn1, HSIC score:nan
Cross HSIC score:nan
Denom:nan
Layer 1:layer2.0.conv2.avgpool, HSIC score:nan
Layer 1:layer1.0.relu1, HSIC score:nan
Cross HSIC score:nan
Denom:nan
Layer 1:layer2.0.conv2.avgpool, HSIC score:nan
Layer 1:layer1.0.bn2, HSIC score:1.3307418100794166e-07
Cross HSIC score:nan
Denom:nan
Layer 1:layer2.0.conv2.avgpool, HSIC score:nan
Lay

Layer 1:layer2.0.bn2, HSIC score:685302096068608.0
Layer 1:layer2.0.conv2.avgpool, HSIC score:nan
Cross HSIC score:inf
Denom:nan
Layer 1:layer2.0.relu2, HSIC score:685302096068608.0
Layer 1:layer1.0.bn1, HSIC score:nan
Cross HSIC score:inf
Denom:nan
Layer 1:layer2.0.relu2, HSIC score:685302096068608.0
Layer 1:layer1.0.relu1, HSIC score:nan
Cross HSIC score:inf
Denom:nan
Layer 1:layer2.0.relu2, HSIC score:685302096068608.0
Layer 1:layer2.0.conv2.avgpool, HSIC score:nan
Cross HSIC score:inf
Denom:nan
Layer 1:layer2.0.bn3, HSIC score:0.002746131969615817
Layer 1:layer1.0.bn1, HSIC score:nan
Cross HSIC score:inf
Denom:nan
Layer 1:layer2.0.bn3, HSIC score:0.002746131969615817
Layer 1:layer1.0.relu1, HSIC score:nan
Cross HSIC score:inf
Denom:nan
Layer 1:layer2.0.bn3, HSIC score:0.002746131969615817
Layer 1:layer2.0.conv2.avgpool, HSIC score:nan
Cross HSIC score:inf
Denom:nan
Layer 1:layer2.0.relu3, HSIC score:0.002746131969615817
Layer 1:layer1.0.bn1, HSIC score:nan
Cross HSIC score:inf
Deno

Layer 1:layer3.1.bn1, HSIC score:0.004280341789126396
Layer 1:layer1.0.bn1, HSIC score:nan
Cross HSIC score:inf
Denom:nan
Layer 1:layer3.1.bn1, HSIC score:0.004280341789126396
Layer 1:layer1.0.relu1, HSIC score:nan
Cross HSIC score:inf
Denom:nan
Layer 1:layer3.1.bn1, HSIC score:0.004280341789126396
Layer 1:layer2.0.conv2.avgpool, HSIC score:nan
Cross HSIC score:inf
Denom:nan
Layer 1:layer3.1.relu1, HSIC score:0.004280341789126396
Layer 1:layer1.0.bn1, HSIC score:nan
Cross HSIC score:inf
Denom:nan
Layer 1:layer3.1.relu1, HSIC score:0.004280341789126396
Layer 1:layer1.0.relu1, HSIC score:nan
Cross HSIC score:inf
Denom:nan
Layer 1:layer3.1.relu1, HSIC score:0.004280341789126396
Layer 1:layer2.0.conv2.avgpool, HSIC score:nan
Cross HSIC score:inf
Denom:nan
Layer 1:layer3.1.bn2, HSIC score:0.008702298626303673
Layer 1:layer1.0.bn1, HSIC score:nan
Cross HSIC score:inf
Denom:nan
Layer 1:layer3.1.bn2, HSIC score:0.008702298626303673
Layer 1:layer1.0.relu1, HSIC score:nan
Cross HSIC score:inf
De

Layer 1:layer3.4.bn3, HSIC score:0.0034912151750177145
Layer 1:layer1.0.bn1, HSIC score:nan
Cross HSIC score:inf
Denom:nan
Layer 1:layer3.4.bn3, HSIC score:0.0034912151750177145
Layer 1:layer1.0.relu1, HSIC score:nan
Cross HSIC score:inf
Denom:nan
Layer 1:layer3.4.bn3, HSIC score:0.0034912151750177145
Layer 1:layer2.0.conv2.avgpool, HSIC score:nan
Cross HSIC score:inf
Denom:nan
Layer 1:layer3.4.relu3, HSIC score:0.0034912151750177145
Layer 1:layer1.0.bn1, HSIC score:nan
Cross HSIC score:inf
Denom:nan
Layer 1:layer3.4.relu3, HSIC score:0.0034912151750177145
Layer 1:layer1.0.relu1, HSIC score:nan
Cross HSIC score:inf
Denom:nan
Layer 1:layer3.4.relu3, HSIC score:0.0034912151750177145
Layer 1:layer2.0.conv2.avgpool, HSIC score:nan
Cross HSIC score:inf
Denom:nan
Layer 1:layer4.0.bn1, HSIC score:0.0038742106407880783
Layer 1:layer1.0.bn1, HSIC score:nan
Cross HSIC score:inf
Denom:nan
Layer 1:layer4.0.bn1, HSIC score:0.0038742106407880783
Layer 1:layer1.0.relu1, HSIC score:nan
Cross HSIC scor

In [ ]:
import pandas as pd
import seaborn as sns

In [ ]:
import matplotlib.ticker as ticker

In [ ]:
hsicData = {
    'L1' : col1,
    'L2' : col2,
    'hsic':hsicScoreList
}

hsicDataFrame = pd.DataFrame(data=hsicData)
hsicDataFrame = hsicDataFrame.pivot("L1", "L2", "hsic")

In [ ]:
plt.figure(figsize=(15,10))
ax = sns.heatmap(hsicDataFrame.T)
plt.xlabel("Layers "+LABEL,fontsize='36')
plt.ylabel("Layers "+LABEL,fontsize='36')
ax.tick_params(axis='both', which='major', labelsize=20)
ax.set_xticks(ax.get_xticks()[::4])
ax.set_yticks(ax.get_yticks()[::4])
ax.invert_yaxis()
plt.savefig(LABEL+"_"+TYPE+'.pdf',fmt='pdf',bbox_inches='tight')